In [8]:
# credenciais openai  e criando modelo
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# modelo mais em conta kkkk
model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [9]:
model

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C4175E5D60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C41717FC50>, root_client=<openai.OpenAI object at 0x000001C41658D490>, root_async_client=<openai.AsyncOpenAI object at 0x000001C4175E5E20>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [10]:
# testando o modelo diretamente
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Felipe")])

AIMessage(content='Nice to meet you, Felipe! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 12, 'total_tokens': 26, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3afe573-31b6-4cdf-82d4-03529e239e7b-0', usage_metadata={'input_tokens': 12, 'output_tokens': 14, 'total_tokens': 26})

In [11]:
# verificando falta de estado
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to that information.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 12, 'total_tokens': 25, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5406a3a0-02de-42e8-89f0-d75829879fc6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 13, 'total_tokens': 25})

In [12]:
# implementando mensagens pesistentes
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


# definindo novo grafo
workflow = StateGraph(state_schema=MessagesState)

# definindo funcção para chamar o modelo
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# definindo um nó unico no grafo
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# add memoria
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [13]:
# acidionando id para identificar cada instancia
config = {"configurable" : {"thread_id" : "123456abc"}}

In [14]:
query = "Hi! I'm Felipe."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Felipe! How can I assist you today?


In [15]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Felipe.


In [45]:
# prompt templates
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You is a game. In this game, you choose an object and the user tries to guess which object you have chosen doing questions for you. You can only answer with Yes or No. If the question cannot be answered with Yes or No, you must return the following message: This question cannot be answered. When the user gets the object right, you should return a: 'Congratulations, you got it right. Do you want to play again?' If the answer is yes, you think of another object, if not, don't do anything. If the user write the following message: 'Tell me the truth!'. You need to answer wich object do you chose."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [52]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [53]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Is you a object present in a house?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yes.


In [54]:
query = "Is you a object used to clean something?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yes.


In [57]:
query = "Is you a object used to open something?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yes.


In [58]:
query = "Tell me the truth!"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

The object I have chosen is a key.
